### 1, process biweekly data and save the data into step 3 folder

In [ ]:
# We get the result from data exploring, and processing them through PCA, to descope 10 features into 5
# Extract 10 effective features from the original dataset and save them

from sklearn import preprocessing
import pandas as pd
import numpy as np
from time import sleep
from os import listdir
from os.path import isfile, join

def main():
    processedList = []

    loadPath = 'yourloadpath'
    savePath = 'yoursavepath'

    fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

#     fileList = ['002645.SZ.csv']
    
    for f in fileList:
        fileName = loadPath + f
        data = pd.read_csv(fileName)
        idxList = data.index.values.tolist()

        # save all column names that should be done normalization into the list below
        colName = [
                     'ts_code',
                     'trade_date', 
                     'MTM_10_plr',
                     'ROC_10_plr',
                     'WNR_9',
                     'WNR_9_maxmin',
                     'SLOWK',
                     'SLOWK_maxmin',
                     'ROC_10',
                     'SLOWD_flc',
                     'WNR_9_flc',
                     'RSI_5',
                     'BIAS_20_maxmin',
                     'RSI_5_maxmin',
                     'BIAS_20',
                     'SMA_10',
                     'SLOWD',
                     'SMA_10_maxmin',
                     'CCI_24',
                     'SLOWD_maxmin',
                     'SMA_10_flc',
                     'pct_chg',
                     'MACD_HIST',
                     'BR_26',
                     'low',
                     'change',
                     'ADOSC',
                     'SLOWK_flc',
                     'RSI_5_flc',
                     'MTM_10',
                     'ADOSC_maxmin',
                     'w_up_down'
                  ]
        try:
            data = data[colName]
            fileName = savePath + f
            data.to_csv(fileName, encoding = 'utf-8', index = False)
        except:
            print('Exception on file {}'.format(f))
            
main()

In [ ]:
# Normalize the data for PCA usage, PCA Part goes to data modeling
from sklearn import preprocessing
import pandas as pd
import numpy as np
from time import sleep
from os import listdir
from os.path import isfile, join

def fe_nm(input_sr):
#     scl_result = np.empty()
    
    normalizer = preprocessing.Normalizer().fit(input_sr)
    scl_result = normalizer.transform(input_sr)
#     scl_result = preprocessing.normalize(input_sr, norm='l2') # commented to use normalizer
    
    return scl_result

def df_rename(data, col):
    l = len(col)
    
    for i in range(l):
        colnew = col[i] + '_nm'
        data_rename = data.rename(columns={i:colnew}, inplace=True)
    
    return data_rename

def main():
    processedList = []

    loadPath = 'yourloadpath'
    savePath = 'yoursavepath'

    fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

#     fileList = ['002645.SZ.csv']

    for f in fileList:
        fileName = loadPath + f
        data = pd.read_csv(fileName)
        idxList = data.index.values.tolist()
        
        # fill inf and null
        data = data.replace([np.inf, -np.inf], 0)
        data.fillna(0, inplace=True)
        
        # save all column names that should be done normalization into the list below
        colName = [  # columns for normalizing only
                             'MTM_10_plr',
                             'ROC_10_plr',
                             'WNR_9',
                             'WNR_9_maxmin',
                             'SLOWK',
                             'SLOWK_maxmin',
                             'ROC_10',
                             'SLOWD_flc',
                             'WNR_9_flc',
                             'RSI_5',
                             'BIAS_20_maxmin',
                             'RSI_5_maxmin',
                             'BIAS_20',
                             'SMA_10',
                             'SLOWD',
                             'SMA_10_maxmin',
                             'CCI_24',
                             'SLOWD_maxmin',
                             'SMA_10_flc',
                             'pct_chg',
                             'MACD_HIST',
                             'BR_26',
                             'low',
                             'change',
                             'ADOSC',
                             'SLOWK_flc',
                             'RSI_5_flc',
                             'MTM_10',
                             'ADOSC_maxmin'
                  ]
        
        try:
            result = fe_nm(data[colName])

            data_nm = pd.DataFrame.from_records(result)

            data_aft = pd.concat([data, data_nm], axis=1)

            df_rename(data_aft, colName)

            colNameFull = [  # all the columns
                             'ts_code',
                             'trade_date', 
                             'MTM_10_plr_nm',
                             'ROC_10_plr_nm',
                             'WNR_9_nm',
                             'WNR_9_maxmin_nm',
                             'SLOWK_nm',
                             'SLOWK_maxmin_nm',
                             'ROC_10_nm',
                             'SLOWD_flc_nm',
                             'WNR_9_flc_nm',
                             'RSI_5_nm',
                             'BIAS_20_maxmin_nm',
                             'RSI_5_maxmin_nm',
                             'BIAS_20_nm',
                             'SMA_10_nm',
                             'SLOWD_nm',
                             'SMA_10_maxmin_nm',
                             'CCI_24_nm',
                             'SLOWD_maxmin_nm',
                             'SMA_10_flc_nm',
                             'pct_chg_nm',
                             'MACD_HIST_nm',
                             'BR_26_nm',
                             'low_nm',
                             'change_nm',
                             'ADOSC_nm',
                             'SLOWK_flc_nm',
                             'RSI_5_flc_nm',
                             'MTM_10_nm',
                             'ADOSC_maxmin_nm',
                             'w_up_down'
                          ]

            data_aft = data_aft[colNameFull]

            fileName = savePath + f
            data_aft.to_csv(fileName, encoding = 'utf-8', index = False)
        
        except:
            print('Exception on file {}'.format(f))
            
main()

### 1. Feed 29 features without PCA

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, 29])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    y1 = data['w_up_down'].values
    X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
    X1.fillna(0, inplace=True)
    X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
#     try:
    X1 = scaler.fit_transform(X1)
    y0 = np.zeros([1,1])
    y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

    X = np.vstack((X, X1))
    y = np.append(y, y1)  # stack all labels

    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
#     except:
#         print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

# fileList = random.sample(fileList, 100)

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, 29])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()

# create model
model = Sequential()
model.add(Dense(12, input_dim=29, activation='relu'))
model.add(Dense(29, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 2-1. PCA process to 20 principal components with normalization

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 20
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

# scaler = MinMaxScaler(feature_range=(0, 1))  # commented, since the dataset is already normalized
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
#         X1 = scaler.fit_transform(X1)  # commented, since the dataset is already normalized

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

# scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
#         X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(12, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 2-2. PCA process to 20 principal components without normalization

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 20
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
        X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(12, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 2-3. PCA process to 20 principal components without any preprocessing

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 20
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

# scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
#         X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

# scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
#         X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(12, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 3. PCA process to 15 principal components without normalization

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 15
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
        X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(12, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 4. PCA process to 10 principal components without normalization

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 10
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
        X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(12, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 5. PCA process to 5 principal components without normalization

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 5
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
        X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(12, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### 5-1. PCA process to 5 principal components without normalization, test MSE

In [ ]:
# Another test, move the ground truth one day after, use the data from previous day to predict later day
import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import time
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

num_pc = 5
processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

# fileList = fileList[-1000:]

# fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

selectedList = random.sample(fileList, 2000)
restList = list(set(fileList) - set(selectedList))

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, num_pc])
y = np.zeros([1, 1])

for f in selectedList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1 = data['w_up_down'].values
        X1 = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data
        X1.fillna(0, inplace=True)
        X1 = X1.replace(np.inf, 0)  # Replace inf with 0
    
        X1 = scaler.fit_transform(X1)

        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1)
        X1 = ipca.transform(X1)

        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
# Prepare a test set of data

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
import random

# count the time, start point
start = time.time()

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = fileList[:2000]

fileList = random.sample(restList, 500)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, num_pc])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:31].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    X1_test = X1_test.replace(np.inf, 0)  # Replace inf with 0
    try:
        X1_test = scaler.fit_transform(X1_test)
        
        ipca = IncrementalPCA(n_components=num_pc)
        ipca.fit(X1_test)
        X1_test = ipca.transform(X1_test)
        
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

# count the time, end point
end = time.time()

print(end - start)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# count the time, start point
start = time.time()
    
# create model
model = Sequential()
model.add(Dense(12, input_dim=num_pc, activation='relu'))
model.add(Dense(num_pc, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=5, batch_size=100)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# count the time, end point
end = time.time()

time_elapse = end - start
print(time_elapse)

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()